**Support code for Exploratory Report** <br>
https://datastudio.google.com/u/0/reporting/f8bd5e2a-589c-4344-9cfc-36ef1fa471b4

In [1]:
import logging
import pandas as pd
import numpy as np
import pandas_gbq
from datetime import datetime
from web3 import Web3
from google.oauth2 import service_account

from uniswap_utils import getPoolPrice #https://github.com/amantay-a/misc/blob/master/uniswap_utils.py

token_dict = {'ETH':'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', #WETH
              'BTC':'0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', #WBTC
              'USDC':'0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
               'DAI':'0x6B175474E89094C44Da98b954EedeAC495271d0F',
             '1INCH':'0x111111111117dC0aa78b770fA6A738034120C302',
              'AAVE':'0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9',
              'COMP':'0xc00e94Cb662C3520282E6f5717214004A7f26888',
               'DPI':'0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b',
               'FEI':'0x956F47F50A910163D8BF957Cf5846D573E7f87CA',
              'LINK':'0x514910771AF9Ca656af840dff83E8264EcF986CA',
               'SNX':'0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F',
               'UNI':'0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
               'YFI':'0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e',
              'LUNA':'0xd2877702675e6cEb975b4A1dFf9fb7BAF4C91ea9', #WLUNA
               'FTM':'0x4E15361FD6b4BB609Fa63C81A2be19d873717870',
             'SUSHI':'0x6B3595068778DD592e39A122f4f5a5cF09C90fE2',
               'CRV':'0xD533a949740bb3306d119CC777fa900bA034cd52',
               'CVX':'0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B',
               'LDO':'0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32',
             'STETH':'0xDFe66B14D37C77F4E9b180cEb433d1b164f0281D',
              'FRAX':'0x853d955aCEf822Db058eb8505911ED77F175b99e',
              'LUSD':'0x5f98805A4E8be255a32880FDeC7F6728C6568bA0',
              'SUSD':'0x57Ab1ec28D129707052df4dF418D58a2D46d5f51',
              'GUSD':'0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd',
              'USDT':'0xdAC17F958D2ee523a2206206994597C13D831ec7'
            }
# To be able to call historical prices, RPC must support archive mode, suggest using alchemy as it provide archive rpc for free
RPCEndpoint = ''
assert RPCEndpoint, "Set up Archive Endpoint"

w3 = Web3(Web3.HTTPProvider(RPCEndpoint, request_kwargs={'timeout': 20}))
print(f'Blockchain connected: {w3.isConnected()}')

Blockchain connected: True


In [2]:
twapwindows =  [1, 60*10, 60*20]
poolfee = [100, 500, 3000, 10000] # 0.01%, 0.05%, 0.3%, 1.00%

def get_uni_price(w3, tokenA, tokenB, block, TWAPWindows, PoolFees):
    
    price_twap = getPoolPrice(w3, 
                              tokenA, 
                              tokenB,
                              TWAPWindows = twapwindows,
                              blocks = [block],
                              PoolFees = poolfee
                             )
    
    #print(price_twap)
    prices = []    
    ret = {}
    for w in TWAPWindows:
        for i, f in enumerate(PoolFees):
            price = price_twap[block][i]['twap'][w]
            #print('price', price)
            if price:
                prices.append(price)    
        if prices:
            median_price = np.percentile(prices,50, interpolation ='lower')
            ret.update({'twap_'+str(w):median_price}) 
        else:
            ret.update({'twap_'+str(w):None})
    return ret, price_twap

In [3]:
credentials = service_account.Credentials.from_service_account_file(
    'gearbox-336415-5ed144668529.json',
)
gcp_project_id = 'gearbox-336415'

bq_select = '''with u as (select ticker, max(blockNumber) as twap_max_block from gearbox.uniswap_prices group by ticker) 
               select 
                   cl.ticker, cl.blockNumber, cl.updated_at, cl.price_decimal as price_chainlink
               from 
                   gearbox.oracle_price_history cl
                   left join u on u.ticker = cl.ticker
               where cl.base !='usd'
                   and cl.blockNumber > coalesce(twap_max_block,0)
               order by cl.ticker, cl.blockNumber

            '''
df_cl = pandas_gbq.read_gbq(bq_select, 
                             project_id=gcp_project_id,
                             progress_bar_type = None,)
display(df_cl)


,ticker,blockNumber,updated_at,price_chainlink
0,1inch-eth,14685048.0,2022-04-30 10:24:37+00:00,0.000459
1,1inch-eth,14685087.0,2022-04-30 10:31:56+00:00,0.000469
2,1inch-eth,14685897.0,2022-04-30 13:38:40+00:00,0.000459
3,1inch-eth,14686262.0,2022-04-30 14:58:46+00:00,0.000450
4,1inch-eth,14687226.0,2022-04-30 18:40:25+00:00,0.000440
...,...,...,...,...
428,yfi-eth,14701786.0,2022-05-03 01:37:37+00:00,6.108292
429,yfi-eth,14704748.0,2022-05-03 12:51:59+00:00,6.171111
430,yfi-eth,14705728.0,2022-05-03 16:32:04+00:00,6.233097
431,yfi-eth,14705851.0,2022-05-03 17:00:17+00:00,6.295502


In [4]:
df = df_cl
df['tokenA'] = df['ticker'].apply(lambda x: token_dict[x.split('-')[0].upper()])
df['tokenB'] = df['ticker'].apply(lambda x: token_dict[x.split('-')[1].upper()])
df

,ticker,blockNumber,updated_at,price_chainlink,tokenA,tokenB
0,1inch-eth,14685048.0,2022-04-30 10:24:37+00:00,0.000459,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
1,1inch-eth,14685087.0,2022-04-30 10:31:56+00:00,0.000469,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
2,1inch-eth,14685897.0,2022-04-30 13:38:40+00:00,0.000459,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
3,1inch-eth,14686262.0,2022-04-30 14:58:46+00:00,0.000450,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
4,1inch-eth,14687226.0,2022-04-30 18:40:25+00:00,0.000440,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
...,...,...,...,...,...,...
428,yfi-eth,14701786.0,2022-05-03 01:37:37+00:00,6.108292,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
429,yfi-eth,14704748.0,2022-05-03 12:51:59+00:00,6.171111,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
430,yfi-eth,14705728.0,2022-05-03 16:32:04+00:00,6.233097,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
431,yfi-eth,14705851.0,2022-05-03 17:00:17+00:00,6.295502,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2


In [5]:
twapwindow_cols = ['twap_'+str(x) for x in twapwindows]
df[twapwindow_cols] = None
cnt = 0
print(datetime.utcnow(),'Start')
for ticker in df['ticker'].unique():
    df_ticker = df[df['ticker']==ticker].copy()
    print(datetime.utcnow(),f'{cnt}/{len(df)} ({ticker})..')
    for x in df_ticker.itertuples():
        prices, raw = get_uni_price(w3, x.tokenA, x.tokenB, int(x.blockNumber), twapwindows, poolfee)
        prices = list(prices.values())
        df_ticker.loc[(df['blockNumber'] == x.blockNumber)&(df['ticker'] == x.ticker), twapwindow_cols+['raw']] = prices + [str(raw)]
        cnt+=1
        if cnt%100==0:
            print(datetime.utcnow(),f'{cnt}/{len(df)} ({ticker})..')
    df_ticker[twapwindow_cols] = df_ticker[twapwindow_cols].astype('float64')
    pandas_gbq.to_gbq(df_ticker, 
                      'gearbox.uniswap_prices',
                      project_id=gcp_project_id,
                      if_exists = 'append', #replace,
                      progress_bar = False)
display(df)

2022-05-03 23:22:58.199657 Start
2022-05-03 23:22:58.201272 0/433 (1inch-eth)..
2022-05-03 23:23:23.982801 19/433 (aave-eth)..
2022-05-03 23:23:43.968248 33/433 (btc-eth)..
2022-05-03 23:23:58.395673 37/433 (comp-eth)..
2022-05-03 23:24:15.015726 46/433 (crv-eth)..
2022-05-03 23:24:34.317019 58/433 (cvx-eth)..
2022-05-03 23:24:59.424930 78/433 (dai-eth)..
2022-05-03 23:25:30.564524 97/433 (dpi-eth)..
2022-05-03 23:25:35.031481 100/433 (dpi-eth)..
2022-05-03 23:25:45.005273 104/433 (fei-eth)..
2022-05-03 23:26:00.420254 110/433 (frax-eth)..
2022-05-03 23:26:12.679447 115/433 (ftm-eth)..
2022-05-03 23:26:41.883365 134/433 (gusd-eth)..
2022-05-03 23:26:55.857933 141/433 (ldo-eth)..
2022-05-03 23:27:23.396574 164/433 (link-eth)..
2022-05-03 23:27:54.817703 193/433 (luna-eth)..
2022-05-03 23:28:03.238008 200/433 (luna-eth)..
2022-05-03 23:28:11.567963 202/433 (lusd-eth)..
2022-05-03 23:29:56.943972 299/433 (snx-eth)..
2022-05-03 23:29:59.226921 300/433 (snx-eth)..
2022-05-03 23:30:28.784775

,ticker,blockNumber,updated_at,price_chainlink,tokenA,tokenB,twap_1,twap_600,twap_1200
0,1inch-eth,14685048.0,2022-04-30 10:24:37+00:00,0.000459,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
1,1inch-eth,14685087.0,2022-04-30 10:31:56+00:00,0.000469,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
2,1inch-eth,14685897.0,2022-04-30 13:38:40+00:00,0.000459,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
3,1inch-eth,14686262.0,2022-04-30 14:58:46+00:00,0.000450,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
4,1inch-eth,14687226.0,2022-04-30 18:40:25+00:00,0.000440,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
...,...,...,...,...,...,...,...,...,...
428,yfi-eth,14701786.0,2022-05-03 01:37:37+00:00,6.108292,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
429,yfi-eth,14704748.0,2022-05-03 12:51:59+00:00,6.171111,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
430,yfi-eth,14705728.0,2022-05-03 16:32:04+00:00,6.233097,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
431,yfi-eth,14705851.0,2022-05-03 17:00:17+00:00,6.295502,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None


# Exploratory Report
https://datastudio.google.com/u/0/reporting/f8bd5e2a-589c-4344-9cfc-36ef1fa471b4